<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/Generate_text_from_local_prompt_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generative AI - Aumented with documents

Incorporate multiple docs

Evaluating different prompt templates

In [ ]:
!pip install -Uq gpt4all sentence-transformers cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import util, SentenceTransformer
import cohere
from gpt4all import GPT4All
import sqlite3
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
co = cohere.Client("5xvQOPxJqxpUX03AQRONJ1MJTG9knBBTCMHyFjFg")

In [ ]:
sent_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2');

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#model = GPT4All(model_path = "e:\Downloads", model_name="Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf")
## Not sure where collab stores models. You can upload a model to your drive and then
## load it e.g. above
gen_model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")

Downloading: 100%|██████████| 1.98G/1.98G [00:44<00:00, 44.8MiB/s]
Verifying: 100%|██████████| 1.98G/1.98G [00:04<00:00, 451MiB/s]


In [ ]:
## 100 tokens is ~ 3 sentences
DB = "/content/drive/MyDrive/12-4-24-all-mini-token-100.db"

def find_top_k_relevance(question=["No question"], model=None, n=20, db=DB):
  q = model.encode(question)
  scores = {}
  conn = sqlite3.connect(db)

# Create a cursor object
  cursor = conn.cursor()
  # load row by row
  cursor.execute('SELECT * FROM documents')
  for row in cursor:
    scores[row[0]] = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()

  return dict(sorted(scores.items(), key = lambda x: x[1], reverse = True)[:n])


In [ ]:
def get_relevant_docs(top_k):
    top_docs = list(top_k.keys())

    conn = sqlite3.connect(DB)
    docs = []
    # Create a cursor object
    cursor = conn.cursor()
    # load row by row
    # have to do the comprehension to unpakc inside and f-string
    # this doesn't get then in order
    for i in top_docs:
        docs.append(cursor.execute(f"""SELECT sentence FROM documents
                    WHERE id = {i}""").fetchone())

    docs = [i[0] for i in docs]
    results = co.rerank(model = "rerank-english-v3.0", query=q, documents=docs, top_n=4, return_documents=True)
    relevant_ids = []
    # relevant doc ids
    rel_documents = []
    # texts for prompt
    prompt_candidates = []

    for idx, r in enumerate(results.results):
        # only get the best ones..cohere seems to rank them higher than with cosine so 0.4 should be good
        if r.relevance_score > 0.7:
            relevant_ids.append(r.index)

    db_ids = [top_docs[i] for i in relevant_ids]

    conn = sqlite3.connect(DB)
    cursor = conn.cursor()

    for i in db_ids:
        x = cursor.execute(f"""SELECT * FROM documents
                        WHERE id = {i}""").fetchone()

        rel_documents.append(x[2])
        prompt_candidates.append(x[3])

    conn.close()

    return set(rel_documents), prompt_candidates

### Evaluation

Plan is to ask the LLM with and without the context then give them ratings based on some of these:

Answer Relevancy: How relevant is the answer to the question at hand?
For example, if you ask: “What are the ingredients in a peanut butter and jelly sandwich and how do you make it?" and the answer is "You need peanut butter for a peanut butter and jelly sandwich," this answer would have low relevancy. It only provides part of the needed ingredients and doesn't explain how to make the sandwich.‍

Faithfulness: How factually accurate is the answer given the context?
You can mark an answer as faithful if all the claims that are made in the answer can be inferred from the given context. This can be evaluated on a (0,1) scale, where 1 is high faithfulness
‍
Correctness: How accurate is the answer against the ground truth data?
‍
Semantic similarity: How closely does the answer match the context in terms of meaning

In [ ]:
# question list
Questions = [
        "How should I store bananas?",
        "What temperature should I keep milk?",
        "How do I increase shelf life of lettuce?",
        "How does selling fresh produce loose, affect waste?",
        "What is a best-before date for?",
        "What is a use-by date for?",
        "How does removing date labels help?",
        "What are the reasons to not participate in a food waste scheme?",
        "What is the UN's sustainable development goal with respect to food?",
        "Which document relates to primary producers and food waste?",
        "Which document relates to retailers and food waste?",
        "What is the most common use for digestate?",
        "What affects the costs of valorisation?",
        "How many tonnes of food waste are there each year?",
        "What is the benefit of caddy liners?",
        "How to prevent food loss?",
        "How to prevent food waste?",
        "How does composting affect food waste?",
        "What are the main drivers of food waste?",
        "Online shopping and food waste",
        "What is involved with Courthold 2030?",
        "How do you measure in field food waste?"
]

In [ ]:
### Store results in dataframe
cols = ["model", "db", "Question", "retrieved", "default_ans", "context_ans"]

df = pd.DataFrame(columns = cols)

In [ ]:
## models
# mistral-7b-openorca.gguf2.Q4_0.gguf - best overall chat model and commercially licensed
#gen_model = GPT4All("gpt4all-falcon-newbpe-q4_0.gguf")

Downloading: 100%|██████████| 4.21G/4.21G [01:27<00:00, 48.0MiB/s]
Verifying: 100%|██████████| 4.21G/4.21G [00:28<00:00, 150MiB/s]


In [ ]:
default_ans = gen_model.generate(prompt=q, temp=0)

In [ ]:
# orca-mini - orca-mini-3b-gguf2-q4_0.gguf
# nous-hermes-llama2-13b.Q4_0.gguf next
gen_mod = "all-mini"
df = pd.DataFrame(columns = cols)
for q in tqdm(Questions):
    # get top docs from db
    top_k = find_top_k_relevance(question =q, model=sent_model)
    # re rank docs and get best docs
    rel_docs, prompt_candidates = get_relevant_docs(top_k)
    # if no relevant docs
    # Join relevant text together
    retrieved = "\n".join(i for i in prompt_candidates)
    if retrieved == "" or retrieved == "\n":
        retrieved = "No relevant documents found"
    # only need once
    default_ans = gen_model.generate(prompt=q, temp=0)

    # prompt
    x = f"""The original query is as follows: {q}
    We have provided an existing answer: {default_ans}

    We have the opportunity to refine the existing answer (only if needed) with some more context below.
    ------------

    {retrieved}

    ------------
     Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.

     Refined Answer:
     """
    # answer with context
    context_ans = gen_model.generate(prompt=x, temp=0)
    # append to database
    df.loc[len(df.index)] = [gen_mod, DB, q,retrieved , default_ans, context_ans]




  0%|          | 0/22 [00:00<?, ?it/s]

In [58]:
from pprint import pprint

In [ ]:
df

,model,db,Question,retrieved,default_ans,context_ans
0,all-mini,/content/drive/MyDrive/12-4-24-all-mini-token-...,How should I store bananas?,. Visual quality was maintained longer where t...,"\nBananas are best stored in a cool, dry place...","\nBananas are best stored in a cool, dr..."


In [64]:
df.retrieved[0]

'. Visual quality was maintained longer where the fruit was kept in polyethylene bags. It is not appropriate to store bananas in the fridge as they are chilling-sensitive, and their skins become blackened, but the coolest place in the home is the best storage location and the fruit should be kept in bags. Bananas sold in the UK are mostly sourced from Central and South America. They are harvested while still in a pre-climacteric (green) condition and shipped to the UK where\n bags, and could, after further research, act as an alternative to conventional plastic (Ahmed, 2016).  Speed of ripening aside, a further benefit of packaging is to protect bananas from bruising and excessive handling by citizens. Retailers have reported that damage is caused by citizens breaking bunches of loose bananas while selecting the appropriate number of fruits and their preferred level of ripeness (WRAP, 2011). The damage to loose fruits at this stage, however, is potentially offset by the bruising caused

In [ ]:
df["default_ans"][0]

'\nBananas are best stored in a cool, dry place away from direct sunlight. You can keep them in your pantry or refrigerator. If you choose to store them in the refrigerator, make sure to remove any fruit that has browned or softened before storing new ones.'

In [68]:

pprint(df["context_ans"][0])

('\n'
 'Bananas are best stored in a cool, dry place away from direct sunlight. You '
 'can keep them in your pantry or refrigerator. If you choose to store them in '
 'the refrigerator, make sure to remove any fruit that has browned or softened '
 'before storing new ones. It is not appropriate to store bananas in the '
 'fridge as they are chilling-sensitive, and their skins become blackened, but '
 'the coolest place in the home is the best storage location and the fruit '
 'should be kept in bags. Bananas sold in the UK are mostly sourced from '
 'Central and South America. They are harvested while still in a '
 'pre-climacteric (green) condition and shipped to the UK where they are '
 'packaged in polyethylene bags. Retailers have reported that damage is caused '
 'by citizens breaking bunches of loose bananas while selecting the '
 'appropriate number of fruits and their preferred level of ripeness, but this '
 'can be offset by the bruising caused by packaging.')


In [71]:
df.to_csv("/content/drive/MyDrive/all-mini-new-prompt.csv")

## Tried templates


f
"""### System:
            You are an sophisticated AI assistant that helps to answer questions as best you can.

            ### User:
            {q}

            ### System:
            Based on the retrieved information from the PDF, here are the relevant excerpts:

            {retrieved}

            Please answer the user's question, integrating insights from these excerpts and your general knowledge. Limit your answer to Three sentences.

            ### Response:
            """

f"""Context information is below.
            ---------------------
            {retrieved}
            ---------------------
            Given the context information and not prior knowledge, answer the query in three sentences.
            Query: {q}
            Answer:
            """

 x = f"""The original query is as follows: {q}
    We have provided an existing answer: {default_ans}

    We have the opportunity to refine the existing answer (only if needed) with some more context below.
    ------------

    {retrieved}
    
    ------------
     Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.

     Refined Answer:
     """